# Workflow to Validate NISAR L2 Secular Displacement Requirement

**Original code authored by:** David Bekaert, Heresh Fattahi, Eric Fielding, and Zhang Yunjun 

Extensive modifications by Adrian Borsa and Amy Whetter and other NISAR team members 2022

Reorganized and modified by Ekaterina Tymofyeyeva, March 2024

<div class="alert alert-warning">
Both the initial setup (<b>Prep A</b> section) and download of the data (<b>Prep B</b> section) should be run at the start of the notebook. And all subsequent sections NEED to be run in order.
</div>



## Define CalVal Site and directory structure

In [ ]:
# Choose a site and track direction
site='MojaveD173' 

# Define your directory structure - you won't need to change this line
start_directory = '/scratch/nisar-st-calval-solidearth' 

# What dataset are you processing?
dataset = 'ARIA_S1' # For Sentinel-1 testing with aria-tools

# The date and version of this Cal/Val run
today = '20240429'
version = '1'

# The file where you keep your customized list of sites.
custom_sites = '/home/jovyan/my_sites.txt'

# Please enter a name or username that will determine where your outputs are stored
import os
if os.path.exists('/home/jovyan/me.txt'): # if OpenTopo API key already installed
    with open('/home/jovyan/me.txt') as m:
        you = m.readline().strip()
    print('You are', you)
    print('Using this as the name of the directory where your outputs will be stored.')
    print('Directory structure: start_directory / dataset/ requirement / site / you / today / version ')

else:
    print('We need a name or username (determines where your outputs will be stored)')
    print('Directory structure: start_directory / dataset/ requirement / site / you / today / version ')

    you = input('Please type your name:')
    with open ('/home/jovyan/me.txt', 'w') as m: 
        m.write(you)
    

## Table of Contents:
<a id='secular_TOC'></a>

<hr/>

[**1. Generation of Time Series from Interferograms**](#secular_gen_ts)
- [1.1. Validate/Modify Interferogram Network](#secular_validate_network)
- [1.2. Generate Quality Control Mask](#secular_generate_mask)
- [1.3. Reference Interferograms To Common Lat/Lon](#secular_common_latlon)
- [1.4. Invert for SBAS Line-of-Sight Timeseries](#secular_invert_SBAS)

[**2. Optional Corrections**](#secular_opt_correction)
- [2.1. Solid Earth Tide Correction](#secular_solid_earth)
- [2.2. Tropospheric Delay Correction](#secular_tropo_corr)
- [2.3. Phase Deramping ](#secular_phase_deramp)
- [2.4. Topographic Residual Correction ](#secular_topo_corr) 

[**3. Estimate InSAR and GNSS Velocities**](#secular_decomp_ts)
- [3.1. Estimate InSAR LOS Velocities](#secular_insar_vel1)
- [3.2. Find Collocated GNSS Stations](#secular_co_gps)  
- [3.3. Get GNSS Position Time Series](#secular_gps_ts) 
- [3.4. Make GNSS LOS Velocities](#secular_gps_los)
- [3.5. Re-Reference GNSS and InSAR Velocities](#secular_gps_insar)

[**4. NISAR Validation Approach 1: GNSS-InSAR Direct Comparison**](#secular_nisar_validation)
- [4.1. Make Velocity Residuals at GNSS Locations](#secular_make_vel)
- [4.2. Make Double-differenced Velocity Residuals](#secular_make_velres)
- [4.3. Secular Requirement Validation: Method 1](#secular_valid_method1)

[**5. NISAR Validation Approach 2: InSAR-only Structure Function**](#secular_nisar_validation2)
- [5.1. Read Array and Mask Pixels with no Data](#secular_array_mask)
- [5.2. Randomly Sample Pixels and Pair Them Up with Option to Remove Trend](#secular_remove_trend)
- [5.3. Amplitude vs. Distance of Relative Measurements (pair differences)](#secular_M2ampvsdist2)
- [5.4. Bin Sample Pairs by Distance Bin and Calculate Statistics](#secular_M2RelMeasTable)

[**Appendix: Supplementary Comparisons and Plots**](#secular_appendix1)
- [A.1. Compare Raw Velocities](#secular_compare_raw)
- [A.2. Plot Velocity Residuals](#secular_plot_vel)
- [A.3. Plot Double-differenced Residuals](#secular_plot_velres)
- [A.4. GPS Position Plot](#secular_appendix_gps)

<hr/>

# Environment setup

In [1]:
################# You should not need to change any code in this cell ##########################################

#Load Packages
import glob
import os
import subprocess
from datetime import datetime as dt
from pathlib import Path
import json

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from mintpy.cli import view, plot_network
from mintpy.objects import gnss, timeseries
from mintpy.smallbaselineApp import TimeSeriesAnalysis
from mintpy.utils import ptime, readfile, utils as ut
from scipy import signal

from solid_utils.sampling import load_geo, samp_pair, profile_samples, haversine_distance
from solid_utils.plotting import display_validation, display_validation_table

#Set Global Plot Parameters
plt.rcParams.update({'font.size': 12})

################# Set Directories ##########################################
requirement = 'Secular'
work_dir = os.path.join(start_directory,dataset,requirement,site,you,today,'v'+version)
print("Work directory:", work_dir)

gunw_dir = os.path.join(work_dir,'products')
print("   GUNW    dir:", gunw_dir) 

mintpy_dir = os.path.join(work_dir,'MintPy')
print("   MintPy  dir:", mintpy_dir)

### Change to MintPy workdir
if not os.path.exists(mintpy_dir):
    print()
    print('ERROR: Stop! Your MintPy processing directory does not exist for this requirement, site, version, or date of your ATBD run.')
    print('You may need to run the prep notebook first!')
    print()
else:
    os.chdir(mintpy_dir)

vel_file = os.path.join(mintpy_dir, 'velocity.h5')
msk_file = os.path.join(mintpy_dir, 'maskConnComp.h5')  # maskTempCoh.h5 maskConnComp.h5

with open(custom_sites,'r') as fid:
    sitedata = json.load(fid)


ImportError: cannot import name 'gnss' from 'mintpy.objects' (/u/trappist-r0/govorcin/src/tools/Mintpy-fork/MintPy/src/mintpy/objects/__init__.py)

# 1. Generation of Time Series from Interferograms

<a id='secular_validate_network'></a>
## 1.1. Validate/Modify Interferogram Network

Add additional parameters to config_file in order to remove selected interferograms, change minimum coherence, etc.

In [ ]:
config_file = os.path.join(mintpy_dir,site + '.cfg')
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep modify_network'
process = subprocess.run(command, shell=True)

plot_network.main(['inputs/ifgramStack.h5'])

<a id='secular_generate_mask'></a>
## 1.2. Generate Quality Control Mask

Mask files can be can be used to mask pixels in the time-series processing. Below we generate a mask file based on the connected components, which is a metric for unwrapping quality.

In [ ]:
command='generate_mask.py inputs/ifgramStack.h5  --nonzero  -o maskConnComp.h5  --update'
process = subprocess.run(command, shell=True)
view.main(['maskConnComp.h5', 'mask'])

<a id='secular_common_latlon'></a>
## 1.3. Reference Interferograms To Common Lat/Lon

In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep reference_point'
process = subprocess.run(command, shell=True)
os.system('info.py inputs/ifgramStack.h5 | egrep "REF_"');

<a id='secular_invert_SBAS'></a>
## 1.4. Invert for SBAS Line-of-Sight Timeseries

In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep invert_network'
process = subprocess.run(command, shell=True)

<a id='secular_opt_correction'></a>
# 2. Optional Corrections

Phase distortions related to solid earth and ocean tidal effects, as well as those due to temporal variations in the vertical stratification of the atmosphere, can be mitigated using the approaches described below. At this point, it is expected that these corrections will not be needed to validate the mission requirements, but they may be used to produce the highest quality data products. Typically, these are applied to the estimated time series product rather than to the individual interferograms, since they are a function of the time of each radar acquisition.

<a id='secular_solid_earth'></a>
## 2.1. Solid Earth Tide Correction

[MintPy provides functionality for this correction.]

<a id='secular_tropo_corr'></a>
## 2.2. Tropospheric Delay Correction

Optional atmospheric correction utilizes the PyAPS (Jolivet et al., 2011, Jolivet and Agram, 2012) module within GIAnT (or eventually a merged replacement for GIAnT and MintPy). PyAPS is well documented, maintained and can be freely downloaded. PyAPS is included in GIAnT distribution). PyAPS currently includes support for ECMWF’s ERA-Interim, NOAA’s NARR and NASA’s MERRA weather models. A final selection of atmospheric models to be used for operational NISAR processing will be done during Phase C.

[T]ropospheric delay maps are produced from atmospheric data provided by Global Atmospheric Models. This method aims to correct differential atmospheric delay correlated with the topography in interferometric phase measurements. Global Atmospheric Models (hereafter GAMs)... provide estimates of the air temperature, the atmospheric pressure and the humidity as a function of elevation on a coarse resolution latitude/longitude grid. In PyAPS, we use this 3D distribution of atmospheric variables to determine the atmospheric phase delay on each pixel of each interferogram.

The absolute atmospheric delay is computed at each SAR acquisition date. For a pixel a_i at an elevation z at acquisition date i, the four surrounding grid points are selected and the delays for their respective elevations are computed. The resulting delay at the pixel a_i is then the bilinear interpolation between the delays at the four grid points. Finally, we combine the absolute delay maps of the InSAR partner images to produce the differential delay maps used to correct the interferograms.

[MintPy provides functionality for this correction.]

In [ ]:
do_tropo_correction = False
########################################################################
'''
REFERENCE : https://github.com/insarlab/pyaps#2-account-setup-for-era5
Read Section 2 for ERA5 [link above] to create an account on the CDS website.
'''

if do_tropo_correction:
    if not Use_Staged_Data and not os.path.exists(Path.home()/'.cdsapirc'):
        print('NEEDED to download ERA5, link: https://cds.climate.copernicus.eu/user/register')
        UID = input('Please type your CDS_UID:')
        CDS_API = input('Please type your CDS_API:')
        
        cds_tmp = '''url: https://cds.climate.copernicus.eu/api/v2
        key: {UID}:{CDS_API}'''.format(UID=UID, CDS_API=CDS_API)
        os.system('echo "{cds_tmp}" > ~/.cdsapirc; chmod 600 ~/.cdsapirc'.format(cds_tmp = str(cds_tmp)))
    
    command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep correct_troposphere'
    process = subprocess.run(command, shell=True)
    
    view.main(['inputs/ERA5.h5'])
    timeseries_filename = 'timeseries_ERA5.h5'
else:
    timeseries_filename = 'timeseries.h5'

<a id='secular_phase_deramp'></a>
## 2.3. Phase Deramping


In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep deramp'
process = subprocess.run(command, shell=True)

<a id='secular_topo_corr'></a>
## 2.4. Topographic Residual Correction 

[MintPy provides functionality for this correction.]

In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep correct_topography'
process = subprocess.run(command, shell=True)

<a id='secular_decomp_ts'></a>
# 3. Estimate InSAR and GNSS Velocities
The approach that will be used for the generation of NISAR L3 products for Requirements 660 and 663 allows for an explicit inclusion of key basis functions (e.g., Heaviside functions, secular rate, etc.) in the InSAR inversion. Modifications to this algorithm may be identified and implemented in response to NISAR Phase C activities. 

<a id='secular_insar_vel1'></a>
## 3.1. Estimate InSAR LOS Velocities

Given a time series of InSAR LOS displacements, the observations for a given pixel, $U(t)$, can be parameterized as:

$$U(t) = a \;+\; vt \;+\; c_1 cos (\omega_1t - \phi_{1,}) \;+\; c_2 cos (\omega_2t - \phi_2) \;+\; \sum_{j=1}^{N_{eq}} \left( h_j+f_j F_j (t-t_j) \right)H(t - t_j) \;+\; \frac{B_\perp (t)}{R sin \theta}\delta z \;+\; residual$$ 

which includes a constant offset $(a)$, velocity $(v)$, and amplitudes $(c_j)$ and phases $(\phi_j)$ of annual $(\omega_1)$ and semiannual $(\omega_2)$ sinusoidal terms.  Where needed we can include additional complexity, such as coseismic and postseismic processes parameterized by Heaviside (step) functions $H$ and postseismic functions $F$ (the latter typically exponential and/or logarithmic).   $B_\perp(t)$, $R$, $\theta$, and $\delta z$ are, respectively, the perpendicular component of the interferometric baseline relative to the first date, slant range distance, incidence angle and topography error correction for the given pixel. 

Thus, given either an ensemble of interferograms or the output of SBAS (displacement vs. time), we can write the LSQ problem as 

$$ \textbf{G}\textbf{m} = \textbf{d}$$

where $\textbf{G}$ is the design matrix (constructed out of the different functional terms in Equation 2 evaluated either at the SAR image dates for SBAS output, or between the dates spanned by each pair for interferograms), $\textbf{m}$ is the vector of model parameters (the coefficients in Equation 2) and $\textbf{d}$ is the vector of observations.  For GPS time series, $\textbf{G}, \textbf{d}, \textbf{m}$ are constructed using values evaluated at single epochs corresponding to the GPS solution times, as for SBAS InSAR input. 

With this formulation, we can obtain InSAR velocity estimates and their formal uncertainties (including in areas where the expected answer is zero). 

The default InSAR velocity fit in MintPy is to estimate a mean linear velocity $(v)$ in in the equation, which we do below. 

In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep velocity'
process = subprocess.run(command, shell=True)

# load velocity file
insar_velocities,_ = readfile.read(vel_file, datasetName = 'velocity')  # read velocity file
insar_velocities = insar_velocities * 1000.  # convert velocities from m to mm

# set masked pixels to NaN
msk,_ = readfile.read(msk_file)
insar_velocities[msk == 0] = np.nan
insar_velocities[insar_velocities == 0] = np.nan

Now we plot the mean linear velocity fit. The MintPy `view` module automatically reads the temporal coherence mask `maskTempCoh.h5` and applies that to mask out pixels with unreliable velocities (white).

In [ ]:
scp_args = 'velocity.h5 velocity -v -25 25 --colormap RdBu_r --figtitle LOS_Velocity --unit mm/yr -m ' + msk_file
view.main(scp_args.split())

<div class="alert alert-info">
<b>Note :</b> 
Negative values indicates that target is moving away from the radar (i.e., Subsidence in case of vertical deformation).
Positive values indicates that target is moving towards the radar (i.e., uplift in case of vertical deformation). 
</div>

<a id='secular_co_gps'></a>
## 3.2. Find Collocated GNSS Stations

The project will have access to L2 position data for continuous GNSS stations in third-party networks such NSF’s Plate Boundary Observatory, the HVO network for Hawaii, GEONET-Japan, and GEONET-New Zealand, located in target regions for NISAR solid earth calval. Station data will be post-processed by one or more analysis centers, will be freely available, and will have latencies of several days to weeks, as is the case with positions currently produced by the NSF’s GAGE Facility and separately by the University of Nevada Reno. Networks will contain one or more areas of high-density station coverage (2~20 km nominal station spacing over 100 x 100 km or more) to support validation of L2 NISAR requirements at a wide range of length scales.

In [ ]:
# get analysis metadata from InSAR velocity file
insar_metadata = readfile.read_attribute(vel_file)
lat_step = float(insar_metadata['Y_STEP'])
lon_step = float(insar_metadata['X_STEP'])
(S,N,W,E) = ut.four_corners(insar_metadata)
start_date = insar_metadata.get('START_DATE', None).split('T')[0]
end_date = insar_metadata.get('END_DATE', None).split('T')[0]
start_date_gnss = dt.strptime(start_date, "%Y%m%d")
end_date_gnss = dt.strptime(end_date, "%Y%m%d")
#inc_angle = int(float(insar_metadata.get('incidenceAngle', None)))
#az_angle = int(float(insar_metadata.get('azimuthAngle', None)))
# Just testing!
inc_angle = 32.5
az_angle = 0.0

#Set GNSS Parameters
gnss_completeness_threshold = 0.9    #0.9  #percent of data timespan with valid GNSS epochs
gnss_residual_stdev_threshold = 10.  #0.03  #0.03  #max threshold standard deviation of residuals to linear GNSS fit

# search for collocated GNSS stations
site_names, site_lats, site_lons = gnss.search_gnss(SNWE=(S,N,W,E), start_date=start_date, end_date=end_date)
site_names = [str(stn) for stn in site_names]
print("Initial list of {} stations used in analysis:".format(len(site_names)))
print(site_names)

In [ ]:
insar_metadata

<a id='secular_gps_ts'></a>
## 3.3. Get GNSS Position Time Series


In [ ]:
# get daily position solutions for GNSS stations
use_stn = []  #stations to keep
bad_stn = []  #stations to toss
use_lats = [] 
use_lons = []

for counter, stn in enumerate(site_names):
    gnss_obj = gnss.get_gnss_class('UNR')(site = stn, data_dir = os.path.join(mintpy_dir,'GNSS'))
    gnss_obj.open(print_msg=False)
    
    # count number of dates in time range
    dates = gnss_obj.dates
    range_days = (end_date_gnss - start_date_gnss).days
    gnss_count = np.histogram(dates, bins=[start_date_gnss, end_date_gnss])
    gnss_count = int(gnss_count[0])
    
    # for this quick screening check of data quality, we use the constant incidence and azimuth angles 
    # get standard deviation of residuals to linear fit
    disp_los = ut.enu2los(gnss_obj.dis_e, gnss_obj.dis_n, gnss_obj.dis_u, inc_angle, az_angle)
    disp_detrended = signal.detrend(disp_los)
    stn_stdv = np.std(disp_detrended)
   
    # select GNSS stations based on data completeness and scatter of residuals
    disp_detrended = signal.detrend(disp_los)
    if range_days * gnss_completeness_threshold <= gnss_count:
        if stn_stdv > gnss_residual_stdev_threshold:
            bad_stn.append(stn)
        else:
            use_stn.append(stn)
            use_lats.append(site_lats[counter])
            use_lons.append(site_lons[counter])
    else:
        bad_stn.append(stn)

site_names = use_stn
site_lats = use_lats
site_lons = use_lons

# [optional] manually remove additional stations
gnss_to_remove=[]

for i, gnss_site in enumerate(gnss_to_remove):
    if gnss_site in site_names:
        site_names.remove(gnss_site)
    if gnss_site not in bad_stn:
        bad_stn.append(gnss_site)

print("\nFinal list of {} stations used in analysis:".format(len(site_names)))
print(site_names)
print("List of {} stations removed from analysis".format(len(bad_stn)))
print(bad_stn)

<a id='secular_gps_los'></a>
## 3.4. Project GNSS to LOS Velocities

In [ ]:
gnss_velocities = gnss.get_los_obs(insar_metadata, 
                            'velocity', 
                            site_names, 
                            start_date=start_date,
                            end_date=end_date,
                            gnss_comp='enu2los', 
                            redo=True)

# scale site velocities from m/yr to mm/yr
gnss_velocities *= 1000.

print('\n site   vel_los [mm/yr]')
print(np.array([site_names, gnss_velocities]).T)

<a id='secular_gps_insar'></a>
## 3.5. Re-Reference GNSS and InSAR LOS Velocities


In [ ]:
# reference GNSS stations to GNSS reference site
ref_site_ind = site_names.index(sitedata['sites'][site]['gps_ref_site_name'])
gnss_velocities = gnss_velocities - gnss_velocities[ref_site_ind]

# reference InSAR to GNSS reference site
ref_site_lat = float(site_lats[ref_site_ind])
ref_site_lon = float(site_lons[ref_site_ind])
ref_y, ref_x = ut.coordinate(insar_metadata).geo2radar(ref_site_lat, ref_site_lon)[:2]
insar_velocities = insar_velocities - insar_velocities[ref_y, ref_x]

# plot GNSS stations on InSAR velocity field
vmin, vmax = -20, 20
cmap = plt.get_cmap('RdBu_r')

fig, ax = plt.subplots(figsize=[18, 5.5])
cax = ax.imshow(insar_velocities, cmap=cmap, vmin=vmin, vmax=vmax, interpolation='nearest', extent=(W, E, S, N))
cbar = fig.colorbar(cax, ax=ax)
cbar.set_label('LOS velocity [mm/year]')

for lat, lon, obs in zip(site_lats, site_lons, gnss_velocities):
    color = cmap((obs - vmin)/(vmax - vmin))
    ax.scatter(lon, lat, color=color, s=8**2, edgecolors='k')
for i, label in enumerate(site_names):
     plt.annotate(label, (site_lons[i], site_lats[i]), color='black')

out_fig = os.path.abspath('vel_insar_vs_gnss.png')
fig.savefig(out_fig, bbox_inches='tight', transparent=True, dpi=300)

<a id='secular_nisar_validation'></a>
# 4. NISAR Validation Approach 1: GNSS-InSAR Direct Comparison 


In [ ]:
# Define secular requirement
secular_gnss_rqmt = 2  # mm/yr for 3 years of data over length scales of 0.1-50 km
gnss_dist_rqmt = [0.1, 50.0]  # km

# Statistics
n_bins = 10
threshold = 0.683  
#  we assume that the distribution of residuals is Gaussian and 
#  that the threshold represents a 1-sigma limit within which 
#  we expect 68.3% of residuals to lie.

<a id='secular_make_vel'></a>
## 4.1. Make Velocity Residuals at GNSS Locations


In [ ]:
#Set Parameters
pixel_radius = 5   #number of InSAR pixels to average for comparison with GNSS

#Create dictionary with the stations as the key and all their info as an array 
stn_dict = {}

#Loop over GNSS station locations
for i in range(len(site_names)): 
    # convert GNSS station lat/lon information to InSAR x/y grid
    stn_lat = site_lats[i]
    stn_lon = site_lons[i]
    x_value = round((stn_lon - W)/lon_step)
    y_value = round((stn_lat - N)/lat_step)
    
    # get velocities and residuals
    gnss_site_vel = gnss_velocities[i]
    #Caution: If you expand the radius parameter farther than the bounding grid it will break. 
    #To fix, remove the station in section 4 when the site_names list is filtered
    vel_px_rad = insar_velocities[y_value-pixel_radius:y_value+1+pixel_radius, 
                     x_value-pixel_radius:x_value+1+pixel_radius]
    insar_site_vel = np.median(vel_px_rad)
    residual = gnss_site_vel - insar_site_vel

    # populate data structure
    values = [x_value, y_value, insar_site_vel, gnss_site_vel, residual, stn_lat, stn_lon]
    stn = site_names[i]
    stn_dict[stn] = values

# extract data from structure
res_list = []
insar_site_vels = []
gnss_site_vels = []
lat_list = []
lon_list = []
for i in range(len(site_names)): 
    stn = site_names[i]
    insar_site_vels.append(stn_dict[stn][2])
    gnss_site_vels.append(stn_dict[stn][3])
    res_list.append(stn_dict[stn][4])
    lat_list.append(stn_dict[stn][5])
    lon_list.append(stn_dict[stn][6])
num_stn = len(site_names) 
print('Finish creating InSAR residuals at GNSS sites')

<a id='secular_make_velres'></a>
## 4.2. Make Double-Differenced Velocity Residuals


In [ ]:
n_gnss_sites = len(site_names)
diff_res_list = []
stn_dist_list = []

# loop over stations
for i in range(n_gnss_sites-1):
    stn1 = site_names[i]
    for j in range(i + 1, n_gnss_sites):
        stn2 = site_names[j]

        # calculate GNSS and InSAR velocity differences between stations
        gnss_vel_diff = stn_dict[stn1][3] - stn_dict[stn2][3]
        insar_vel_diff = stn_dict[stn1][2] - stn_dict[stn2][2]

        # calculate GNSS vs InSAR differences (double differences) between stations
        diff_res = gnss_vel_diff - insar_vel_diff
        diff_res_list.append(diff_res)

        # get distance (km) between stations using Haversine formula
        # index 5 is lat, 6 is lon
        stn_dist = haversine_distance(stn_dict[stn1][6], stn_dict[stn1][5], stn_dict[stn2][6], stn_dict[stn2][5])
        stn_dist_list.append(stn_dist)

# Write data for statistical tests
gnss_site_dist = np.array(stn_dist_list)
double_diff_rel_measure = np.array(np.abs(diff_res_list))
ndx = np.argsort(gnss_site_dist)

<a id='secular_valid_method1'></a>
## 4.3. Secular Requirement Validation: Method 1


In [2]:
validation_table, fig = display_validation(gnss_site_dist, double_diff_rel_measure,
                                          site, start_date,
                                          end_date, requirement=secular_gnss_rqmt,
                                          distance_rqmt=gnss_dist_rqmt, n_bins=n_bins,
                                          threshold=threshold, sensor='Sentinel-1',
                                          validation_type='secular', validation_data='GNSS')
out_fig = os.path.abspath('secular_insar-gnss_velocity_vs_distance.png')
fig.savefig(out_fig, bbox_inches='tight', transparent=True, dpi=30)

NameError: name 'display_validation' is not defined

<div class="alert alert-warning">
Final result Method 1—Successful when 68% of points below requirements line
</div>


In [ ]:
display_validation_table(validation_table)

<div class="alert alert-warning">
Final result Method 1 table by distance bin—successful when greater than 0.683
</div>


<a id='secular_nisar_validation2'></a>
# 5. NISAR Validation Approach 2: InSAR-only Structure Function

In Validation approach 2, we use a time interval and area where we assume no deformation.

In [ ]:
# define requirement
secular_insar_rqmt = 2  # mm/yr
insar_dist_rqmt = [0.1, 50.0]  # km

# Statistics
n_bins = 10
threshold = 0.683  
#  we assume that the distribution of residuals is Gaussian and 
#  that the threshold represents a 1-sigma limit within which 
#  we expect 68.3% of residuals to lie.

In [ ]:
# plot velocity map
scp_args = 'velocity.h5 velocity -v -20 20 --colormap RdBu_r --figtitle LOS_Velocity --unit mm/yr -m ' + msk_file
view.main(scp_args.split())

<a id='secular_array_mask'></a>
## 5.1. Read Array and Mask Pixels with no Data

In [ ]:
# use the assumed non-earthquake displacement as the insar_displacment for statistics and convert to mm
insar_velocities,_ = readfile.read(vel_file, datasetName = 'velocity')  #read velocity
velStart = sitedata['sites'][site]['download_start_date']
insar_velocities = insar_velocities * 1000.  # convert velocities from m to mm

# set masked pixels to NaN
msk,_ = readfile.read(msk_file)
insar_velocities[msk == 0] = np.nan
insar_velocities[insar_velocities == 0] = np.nan

# display map of data after masking
cmap = plt.get_cmap('RdBu_r')

fig, ax = plt.subplots(figsize=[18, 5.5])
img1 = ax.imshow(insar_velocities, cmap=cmap, vmin=-20, vmax=20, interpolation='nearest', extent=(W, E, S, N))
ax.set_title("Secular \n Date "+velStart)
cbar1 = fig.colorbar(img1, ax=ax)
cbar1.set_label('LOS velocity [mm/year]')

<a id='secular_remove_trend'></a>
## 5.2. Randomly Sample Pixels and Pair Them Up with Option to Remove Trend

In [ ]:
sample_mode = 'profile'  # 'points' or 'profile'
# note that the 'profile' method may take significantly longer

# Collect samples using the specified method
if sample_mode in ['points']:
    X0,Y0 = load_geo(insar_metadata)
    X0_2d,Y0_2d = np.meshgrid(X0,Y0)

    insar_sample_dist, insar_rel_measure = samp_pair(X0_2d, Y0_2d, insar_velocities, num_samples=1000000)

elif sample_mode in ['profile']:
    # Sample grid setup
    length, width = int(insar_metadata['LENGTH']), int(insar_metadata['WIDTH'])
    X = np.linspace(W+lon_step, E-lon_step, width)  # longitudes
    Y = np.linspace(N+lat_step, S-lat_step, length)  # latitudes
    X_coords, Y_coords = np.meshgrid(X, Y)

    # Draw random samples from map (without replacement)
    num_samples = 20000
    
    # Retrieve profile samples
    insar_sample_dist, insar_rel_measure = profile_samples(\
                    x=X_coords.reshape(-1,1),
                    y=Y_coords.reshape(-1,1),
                    data=insar_velocities,
                    metadata=insar_metadata,
                    len_rqmt=insar_dist_rqmt,
                    num_samples=num_samples)

print('Finished sampling')

<a id='secular_M2ampvsdist2'></a>
## 5.3. Amplitude vs. Distance of Relative Measurements (pair differences)

In [ ]:
validation_table, fig = display_validation(insar_sample_dist, insar_rel_measure,
                                           site, start_date,
                                           end_date, requirement=secular_insar_rqmt,
                                           distance_rqmt=insar_dist_rqmt, n_bins=n_bins,
                                           threshold=threshold, sensor='Sentinel-1',
                                           validation_type='secular', validation_data='InSAR')
out_fig = os.path.abspath('secular_insar-only_vs_distance_'+site+'_date'+velStart+'.png')
fig.savefig(out_fig, bbox_inches='tight', transparent=True, dpi=300)

<div class="alert alert-warning">
Final result Method 2—
    68% of points below the requirements line is success
</div>


<a id='secular_M2RelMeasTable'></a>
## 5.4. Bin Sample Pairs by Distance Bin and Calculate Statistics

In [ ]:
display_validation_table(validation_table)

<div class="alert alert-warning">
Final result Method 2 table of distance bins—
    68% (0.683) of points below the requirements line is success
</div>


<a id='secular_appendix1'></a>
# Appendix: Supplementary Comparisons and Plots

<a id='secular_compare_raw'></a>
## A.1. Compare Raw Velocities

In [ ]:
vmin, vmax = -25, 25
plt.figure(figsize=(11,7))
plt.hist(insar_site_vels, range=[vmin, vmax], bins=50, color="green", edgecolor='grey', label='V_InSAR')
plt.hist(gnss_site_vels, range=[vmin, vmax], bins=50, color="orange", edgecolor='grey', label='V_gnss', alpha=0.5)
plt.legend(loc='upper right')
plt.title(f"Velocities \n Date range {start_date}-{end_date} \n Reference stn: {sitedata['sites'][site]['gps_ref_site_name']} \n Number of stations used: {num_stn}")
plt.xlabel('LOS Velocity (mm/year)')
plt.ylabel('N Stations')
plt.ylim(0,20)
plt.show()

<a id='secular_plot_vel'></a>
## A.2. Plot Velocity Residuals


In [ ]:
vmin, vmax = -10, 10
plt.figure(figsize=(11,7))
plt.hist(res_list, bins = 40, range=[vmin,vmax], edgecolor='grey', color="darkblue", linewidth=1, label='V_gnss - V_InSAR (area average)')
plt.legend(loc='upper right')
plt.title(f"Residuals \n Date range {start_date}-{end_date} \n Reference stn: {sitedata['sites'][site]['gps_ref_site_name']} \n Number of stations used: {num_stn}")
plt.xlabel('Velocity Residual (mm/year)')
plt.ylabel('N Stations')
plt.show()

<a id='secular_plot_velres'></a>
## A.3. Plot Double Difference Residuals

In [ ]:
plt.figure(figsize=(11,7))
plt.hist(diff_res_list, range = [vmin, vmax],bins = 40, color = "darkblue",edgecolor='grey',label='V_gnss_(s1-s2) - V_InSAR_(s1-s2)')
plt.legend(loc='upper right')
plt.title(f"Difference Residualts \n Date range {start_date}-{end_date} \n Reference stn: {sitedata['sites'][site]['gps_ref_site_name']} \n Number of stations used: {num_stn}")
plt.xlabel('Double Differenced Velocity Residual (mm/year)')
plt.ylabel('N Stations')
plt.show()

<a id='secular_appendix_gps'></a>
## A.4. GNSS Timeseries Plots


In [ ]:
# grab the time-series file used for time function estimation given the template setup
template = readfile.read_template(os.path.join(mintpy_dir, 'smallbaselineApp.cfg'))
template = ut.check_template_auto_value(template)
insar_ts_file = TimeSeriesAnalysis.get_timeseries_filename(template, mintpy_dir)['velocity']['input']

# read the time-series file
insar_ts, atr = readfile.read(insar_ts_file, datasetName='timeseries')
mask = readfile.read(os.path.join(mintpy_dir, 'maskTempCoh.h5'))[0]
print(f'reading timeseries from file: {insar_ts_file}')

# Get date list
date_list = timeseries(insar_ts_file).get_date_list()
num_date = len(date_list)
date0, date1 = date_list[0], date_list[-1]
insar_dates = ptime.date_list2vector(date_list)[0]

# spatial reference
coord = ut.coordinate(atr)
ref_site = sitedata['sites'][site]['gps_ref_site_name']
ref_gnss_obj = gnss.get_gnss_class('UNR')(site = ref_site, data_dir = os.path.join(mintpy_dir,'GNSS'))
ref_lat, ref_lon = ref_gnss_obj.get_site_lat_lon()
ref_y, ref_x = coord.geo2radar(ref_lat, ref_lon)[:2]
if not mask[ref_y, ref_x]:
    raise ValueError(f'Given reference GNSS site ({ref_site}) is in mask-out unrelible region in InSAR! Change to a different site.')
ref_insar_dis = insar_ts[:, ref_y, ref_x]

# Plot displacements and velocity timeseries at GNSS station locations
num_site = len(site_names)
prog_bar = ptime.progressBar(maxValue=num_site)
for i, site_name in enumerate(site_names):
    prog_bar.update(i+1, suffix=f'{site_name} {i+1}/{num_site}')

    ## read data
    # read GNSS
    gnss_obj = gnss.get_gnss_class('UNR')(site = site_name, data_dir = os.path.join(mintpy_dir,'GNSS'))
    gnss_dates, gnss_dis, _, gnss_lalo = gnss_obj.get_los_displacement(atr, start_date=date0, end_date=date1, ref_site=ref_site)[:4]
    # shift GNSS to zero-mean in time [for plotting purpose]
    gnss_dis -= np.nanmedian(gnss_dis)

    # read InSAR
    y, x = coord.geo2radar(gnss_lalo[0], gnss_lalo[1])[:2]
    insar_dis = insar_ts[:, y, x] - ref_insar_dis
    # apply a constant shift in time to fit InSAR to GNSS
    comm_dates = sorted(list(set(gnss_dates) & set(insar_dates)))
    if comm_dates:
        insar_flag = [x in comm_dates for x in insar_dates]
        gnss_flag = [x in comm_dates for x in gnss_dates]
        insar_dis -= np.nanmedian(insar_dis[insar_flag] - gnss_dis[gnss_flag])

    ## plot figure
    if gnss_dis.size > 0 and np.any(~np.isnan(insar_dis)):
        fig, ax = plt.subplots(figsize=(12, 3))
        ax.axhline(color='grey',linestyle='dashed', linewidth=2)
        ax.scatter(gnss_dates, gnss_dis*100, s=2**2, label="GNSS Daily Positions")
        ax.scatter(insar_dates, insar_dis*100, label="InSAR Positions")
        # axis format
        ax.set_title(f"Station Name: {site_name}") 
        ax.set_ylabel('LOS displacement [cm]')
        ax.legend()
prog_bar.close()
plt.show()